# Setup and Configure DYNAMOS in the created Kubernetes environment in FABRIC 

This Jupyter notebook will configure the Kubernetes environment so that DYNAMOS can run in it.

FABRIC API docs: https://fabric-fablib.readthedocs.io/en/latest/index.html


## Step 1: Configure the Environment Configure the Environment (has to be done once in the Jupyter Hub environment), Create Slice & Setup K8s Cluster

Before running this notebook, you will need to configure your environment using the [Configure Environment](../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook. Note: this has to be done only once in the Jupyter Hub environment (unless configuration is removed/deleted of course).

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

After following all steps of the Configuring Environment notebook, you should be able to run this notebook without additional steps.

Next, you will need to have setup the slice in FABRIC using the [Create Slice](../create_slice.ipynb) notebook.

Finally, you will need to prepare the Kubernetes environment in FABRIC using the [Configure Kubernetes](../k8s-cluster-setup/k8s_setup.ipynb) notebook.

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

## Step 2: Setup the Environment for this Notebook

### Import FABRIC API and other libraries

In [ ]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();


### (Optional): Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity and other settings or available resources.

In [ ]:
# Comment out lists you do not want to view

# List available images that can be used for the nodes
fablib.get_image_names()

### Step 2.2: Configure the parameters and variables
Can be used to set the corresponding slice and other variables used for subsequent cells.

In [ ]:
slice_name = 'DYNAMOS_EnergyEfficiency'
# Nodes:
node1_name = 'k8s-control-plane'
node2_name = 'dynamos-core'
node3_name = 'vu'
node4_name = 'uva'
node5_name = 'surf'
# Network:
network_name = 'NET1'

### Step 2.3: Set Network IPs for nodes
This step sets the network and IPs of the nodes for later usage.

In [ ]:
# Get slice and slice components
# Get slice by name: https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.get_slice
slice = fablib.get_slice(name=slice_name)
# Get the network
network = slice.get_network(name=network_name)
# Get nodes
nodes = slice.get_nodes()

# Define function to get the ips for each node
def get_ip(node):
    interface = node.get_interface(network_name=network_name)
    return interface.get_ip_addr()
# Define function to get interface of the node
def get_interface(node):
    interface = node.get_interface(network_name=network_name)
    return interface.get_device_name()

# Create a dictionary to store node names and their IPs and interfaces
node_ips = {}
node_interfaces = {}

# Populate the dictionary with node names and their corresponding IPs
for node in nodes:
    node_name = node.get_name()
    # Get IPs for each node from the network and set as variables for later usage
    node_ips[node_name] = get_ip(node)
    # Do the same for interface
    # Get IPs for each node from the network and set as variables for later usage
    node_interfaces[node_name] = get_interface(node)

# Print the IPs and interfaces for each node
for node_name, ip in node_ips.items():
    print(f"{node_name} IP: {ip}")
for node_name, interface in node_interfaces.items():
    print(f"{node_name} interface device for Calico (should be the same for each node): {interface}")

## Step 3: TODO: further steps for the experiments here.
SSH commands to go into the VMs terminal can be fetched from the previous notebook: [Configure Kubernetes](../k8s-cluster-setup/k8s_setup.ipynb).

### Step 3.0: Setup Access to IPv4 from IPv6 only FABRIC VMs
See Troubleshooting.md for more explanation about this related issue. In short, the FABRIC VMs have IPv6 access only as their management IP (i.e. main IP). However, not all platforms support IPv6. This is most important for some custom registries outside the big ones like Github, such as "cr.l5d.io", which caused some problems before. Also, this might fix some other connectivity issues that occured.

In [ ]:
# enable nodes to access IPv4-only resources, such as Github,
# even if the control interface is IPv6-only
# See: https://github.com/teaching-on-testbeds/fabric-snippets/blob/main/ipv4-access-from-ipv6.md
from ipaddress import ip_address, IPv6Address
for node in slice.get_nodes():
    # Check if the node has IPv6 management IP
    if type(ip_address(node.get_management_ip())) is IPv6Address:
        # Adds a working public IPv6 DNS server (from SURFnet/AMS-IX) to the systemd-resolved config file:
        # 2a00:1098:2c::1 is a known reliable IPv6 DNS resolver.
        node.execute('echo "DNS=2a00:1098:2c::1" | sudo tee -a /etc/systemd/resolved.conf')
        # Restarts the DNS resolver service to apply the new config
        node.execute('sudo service systemd-resolved restart')
        # Adds a loopback entry for the hostname to /etc/hosts
        # Fixes issues where hostname -s cannot resolve to localhost
        # Common workaround for tools like sudo, ssh, or certain init systems that rely on hostname resolution
        node.execute('echo "127.0.0.1 $(hostname -s)" | sudo tee -a /etc/hosts')
        # Replaces /etc/resolv.conf with the systemd-managed symlink
        # On many systems, /etc/resolv.conf is incorrectly set (or overwritten by cloud-init). This command:
        # Deletes any existing file, and recreates it as a symlink to the correct DNS config managed by systemd-resolved
        node.execute('sudo rm -f /etc/resolv.conf; sudo ln -sv /run/systemd/resolve/resolv.conf /etc/resolv.conf')

### Step 3.1: Install Linkerd in the Cluster

In [ ]:
try:
    # Get the control plane node (node1)
    node1 = slice.get_node(name=node1_name)
    # Upload script file to the node
    file_attributes = node1.upload_file(local_file_path="linkerd.sh", remote_file_path="linkerd.sh")
    # Add necessary permissions and execute the script
    stdout, stderr = node1.execute(f"chmod +x linkerd.sh && ./linkerd.sh")

except Exception as e:
    print(f"Exception: {e}")

### Step 3.2: Install/Configure DYNAMOS
TODO: refer to README.md:
Go to the README.md in this folder of the GitHub project and follow the steps for "TODO:add specific step here". After these steps, you can do the next steps.

TODO: add here further the DYNAMOS installation.